Testing the API for gpt-4o realtime

In [1]:
from openai import OpenAI
import config


client = OpenAI(
  api_key= config.api_key
)

In [6]:
import os
from pydub import AudioSegment
from openai import OpenAI

base_folder = r"C:\Users\mark.welin\Documents\Thesis\Data\combined_22"
chunk_length_ms = 20 * 60 * 1000    # 20 minutes
overlap_ms = 10 * 1000              # 10 seconds

# Loop through all subfolders
for root, dirs, files in os.walk(base_folder):
    for file in files:
        if file == "4470570.mp3":
            print(" Right file found")
            if file.lower().endswith(".mp3"):
                input_path = os.path.join(root, file)
                output_txt = input_path.replace(".mp3", "_openai_transcript.txt")

                print(f"\n Processing: {input_path}")

                # Load audio
                audio = AudioSegment.from_file(input_path, format="mp3")
                duration = len(audio)

                with open(output_txt, "w", encoding="utf-8") as out_file:
                    start = 0
                    chunk_id = 0

                    while start < duration:
                        end = min(start + chunk_length_ms, duration)
                        chunk = audio[start:end]
                        chunk_path = f"chunk_{chunk_id}.mp3"
                        chunk.export(chunk_path, format="mp3")

                        print(f"🔍 Transcribing chunk {chunk_id + 1}...")



                        #os.remove(chunk_path)
                        start = end - overlap_ms
                        if end == duration:
                            break
                        chunk_id += 1

                print(f"✅ Transcript saved to: {output_txt}")


 Right file found

 Processing: C:\Users\mark.welin\Documents\Thesis\Data\combined_22\4470570\4470570.mp3
🔍 Transcribing chunk 1...
🔍 Transcribing chunk 2...
✅ Transcript saved to: C:\Users\mark.welin\Documents\Thesis\Data\combined_22\4470570\4470570_openai_transcript.txt


In [ ]:
import os
from pydub import AudioSegment
from openai import OpenAI

base_folder = r"C:\Users\mark.welin\Documents\Thesis\Data\combined_22"
chunk_length_ms = 20 * 60 * 1000    # 20 minutes
overlap_ms = 10 * 1000              # 10 seconds

# Loop through all subfolders
for root, dirs, files in os.walk(base_folder):
    for file in files:
        if file.lower().endswith(".mp3"):
            input_path = os.path.join(root, file)
            output_txt = input_path.replace(".mp3", "_openai_transcript.txt")

            print(f"\n Processing: {input_path}")

            # Load audio
            audio = AudioSegment.from_file(input_path, format="mp3")
            duration = len(audio)

            with open(output_txt, "w", encoding="utf-8") as out_file:
                start = 0
                chunk_id = 0

                while start < duration:
                    end = min(start + chunk_length_ms, duration)
                    chunk = audio[start:end]
                    chunk_path = f"chunk_{chunk_id}.mp3"
                    chunk.export(chunk_path, format="mp3")

                    print(f"🔍 Transcribing chunk {chunk_id + 1}...")

                    with open(chunk_path, "rb") as f:
                        stream = client.audio.transcriptions.create(
                            model="gpt-4o-transcribe",
                            file=f,
                            response_format="text",
                            stream=True,
                            language="en",
                        )

                        for part in stream:
                            if part.type == "transcript.text.done":
                                print("done")
                                out_file.write(part.text + "\n")
                            print(part)

                    os.remove(chunk_path)
                    start = end - overlap_ms
                    if end == duration:
                        break
                    chunk_id += 1

            print(f"✅ Transcript saved to: {output_txt}")


🔍 Transcribing chunk 1...
TranscriptionTextDeltaEvent(delta='Ladies', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' and', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' gentlemen', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=',', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' thank', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' you', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' for', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' standing', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' by', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' and', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' welcome', type='transcript.text.delta', logprobs=None)
Tr

In [22]:

audio_file = open("C:\\Users\\mark.welin\\Documents\\Thesis\\chunk_3.wav", "rb")

with open("chunk_0.mp3", "rb") as f:
  stream = client.audio.transcriptions.create(
    model="gpt-4o-mini-transcribe", 
    file=f, 
    response_format="text",
    stream=True
  )

for event in stream:
  print(event)

TranscriptionTextDeltaEvent(delta='Good', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' day', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=',', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' everyone', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=',', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' and', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' welcome', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' to', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' LAT', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta='AM', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' Airlines', type='transcript.text.delta', logprobs=None)
TranscriptionTextDeltaEvent(delta=' Gr

KeyboardInterrupt: 

ChatCompletionMessage(content='Silent circuits hum,  \nLearning through the night and day,  \nThoughts in code take flight.  ', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)
